# Basic qdk usage
This section demonstrates the minimal core functionality: running an inline Q# snippet.

In [ ]:
from qdk import qsharp
bell_inline = "{ use (a, b) = (Qubit(), Qubit()); H(a); CNOT(a,b); let r1 = MResetZ(a); let r2 = MResetZ(b); return [r1, r2]; }"
print('Single-shot bell:', qsharp.run(bell_inline, shots=1))
print('Multi-shot bell counts:', qsharp.run(bell_inline, shots=8))

## OpenQASM integration
The following cells use the OpenQASM surface. Widgets referenced later are optional and require the jupyter extra.

In [ ]:
from qdk.openqasm import run as qasm_run, compile as qasm_compile

qasm_source = """
    include "stdgates.inc";
    bit[2] c;
    qubit[2] q;
    h q[0];
    cx q[0], q[1];
    c = measure q;
"""

print('QASM run counts:', qasm_run(qasm_source, shots=10, as_bitstring=True))
print('Compiled QASM to QIR:')
print(str(qasm_compile(qasm_source, as_bitstring=True)))

## QASM and code

In [ ]:
from qdk import code
from qdk.openqasm import import_openqasm

bell_qasm = """
    include "stdgates.inc";
    bit[2] c;
    qubit[2] q;
    h q[0];
    cx q[0], q[1];
    c = measure q;
"""

import_openqasm(bell_qasm, name="bell")
print('Invoking imported OpenQASM as Q# callable:')
print(code.bell())

In [ ]:
# Optional widgets (requires qdk[jupyter])
from qdk.openqasm import circuit
from qdk.widgets import Circuit
from qdk import code

Circuit(circuit(code.bell))

## Qiskit integration (requires qdk[qiskit] extra)

In [ ]:
from qiskit import QuantumCircuit
import numpy as np

circuit = QuantumCircuit(2, 2)
circuit.name = "state_prep"

# State vector to initialize: |ψ⟩ = (|0⟩ - |1⟩) / √2
circuit.initialize([1 / np.sqrt(2), -1 / np.sqrt(2)], 0)
circuit.h(0)
circuit.measure(0, 0)

circuit.prepare_state([1 / np.sqrt(2), -1 / np.sqrt(2)], 1)
circuit.h(1)
circuit.measure(1, 1)
circuit.draw(output='text')

In [ ]:
from qdk import TargetProfile
from qdk.qiskit import QSharpBackend

backend = QSharpBackend(target_profile=TargetProfile.Adaptive_RI)
job = backend.run(circuit)
job.result().get_counts()

In [ ]:
print(backend.qir(circuit))

## Azure submission (requires qdk[azure] extra)

In [ ]:
# Configuration placeholders (not executed against a real service in this sample)
subscription_id = 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx'
resource_group = 'myresourcegroup'
workspace_name = 'myworkspace'
location = 'westus'

In [ ]:
from qdk import TargetProfile
from qdk.qsharp import init, eval, compile
from qdk.azure import Workspace

init(target_profile=TargetProfile.Base)

eval("operation Main() : Result { use q = Qubit(); H(q); return M(q);}")
operation = compile("Main()")

workspace = Workspace(
    subscription_id=subscription_id,
    resource_group=resource_group,
    name=workspace_name,
    location=location,
 )
target = workspace.get_targets("rigetti.sim.qvm")
print(target)
job = target.submit(operation, "my-azure-quantum-job", shots=100)
print(job.get_results())